Code to Convert MobileNetv3 model to CoreML package.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
import torch.nn as nn
import coremltools as ct
import numpy as np
from PIL import Image
from torchvision import transforms
import json
import urllib

In [ ]:
torch_model = torchvision.models.mobilenet_v3_large(pretrained=True)
torch_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [ ]:
example_input = torch.rand(1, 3, 224, 224)
traced_model = torch.jit.trace(torch_model, example_input)

In [ ]:
label_url = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
class_labels = urllib.request.urlopen(label_url).read().decode("utf-8").splitlines()
class_labels = class_labels[1:]
assert len(class_labels) == 1000

AssertionError: 

In [ ]:
mean = np.array((0.485, 0.456, 0.406))
std = np.array((0.229, 0.224, 0.225))

scale = 1.0 / (0.226 * 255.0)
bias = -mean / std

image_input = ct.ImageType(
    name="x",
    shape=example_input.shape,
    scale=scale,
    bias=bias,
)

In [ ]:
mlmodel_from_trace = ct.convert(
    traced_model,
    inputs=[image_input],
    classifier_config=ct.ClassifierConfig(class_labels),
    compute_units=ct.ComputeUnit.CPU_ONLY,
)

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 40.62 passes/s]


ValueError: In op 'classify', number of classes must match the size of the tensor corresponding to 'probabilities'.

In [ ]:
mlmodel_from_trace.save("/content/drive/MyDrive/mobilenet_v2.mlpackage")

In [ ]:
img_path = "tomato.png"
img = Image.open(img_path).convert('RGB')
img = img.resize([224, 224], Image.LANCZOS)

In [ ]:
img_np = np.asarray(img).astype(np.float32)
img_np = np.expand_dims(img_np, axis=0)
img_np = np.transpose(img_np, [0, 3, 1, 2])
img_np = img_np / 255.0
img_torch = torch.from_numpy(img_np)
img_torch = torchvision.transforms.Normalize(mean=mean, std=std)(img_torch)

In [ ]:
torch_out = torch_model(img_torch)

torch_out_np = torch_out.detach().numpy().squeeze()
top_3_indices = np.argsort(-torch_out_np)[:3]
print("torch top 3 predictions: ")
for i in range(3):
    idx = top_3_indices[i]
    score_value = torch_out_np[idx]
    class_id = class_labels[idx]
    print("class name: {}, raw score value: {}".format(class_id, score_value))